# 1. Making Predictions

In [1]:
# Making a prediction with coefficients
def predict(row,coefficients):
    yhat = coefficients[0]
    for i in range(len(row) - 1):
        yhat+=coefficients[i+1]*row[i]
    return 1.0/(1.0 + exp(-yhat))

In [3]:
## using above function for small dataset

from math import exp

# test predictions
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]

coef = [-0.406605464, 0.852573316, -1.104746259]

for row in dataset:
    yhat = predict(row,coef)
    print("Expected=%.3f, Predicted=%.3f [%d]" %(row[-1],yhat,round(yhat)))

Expected=0.000, Predicted=0.299 [0]
Expected=0.000, Predicted=0.146 [0]
Expected=0.000, Predicted=0.085 [0]
Expected=0.000, Predicted=0.220 [0]
Expected=0.000, Predicted=0.247 [0]
Expected=1.000, Predicted=0.955 [1]
Expected=1.000, Predicted=0.862 [1]
Expected=1.000, Predicted=0.972 [1]
Expected=1.000, Predicted=0.999 [1]
Expected=1.000, Predicted=0.905 [1]


# 2. Estimating Coefficients


In [4]:
# Estimating logistic regression coefficients using stochastic gradient descent
def coefficients_sgd(train,l_rate,n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            yhat = predict(row,coef)
            error = row[-1] - yhat
            sum_error += error**2
            coef[0] = coef[0] + l_rate * error * yhat * (1.0 - yhat)
            for i in range(len(row)-1):
                coef[i+1] = coef[i+1] + l_rate * error * yhat * (1.0 - yhat)*row[i]
        print('>epoch=%d, l_rate=%.3f, error=%.3f' %(epoch,l_rate,sum_error))
    return coef

In [5]:
# calculating coefficients
l_rate = 0.3
n_epoch = 100
coef = coefficients_sgd(dataset,l_rate,n_epoch)
print(coef)

>epoch=0, l_rate=0.300, error=2.217
>epoch=1, l_rate=0.300, error=1.613
>epoch=2, l_rate=0.300, error=1.113
>epoch=3, l_rate=0.300, error=0.827
>epoch=4, l_rate=0.300, error=0.623
>epoch=5, l_rate=0.300, error=0.494
>epoch=6, l_rate=0.300, error=0.412
>epoch=7, l_rate=0.300, error=0.354
>epoch=8, l_rate=0.300, error=0.310
>epoch=9, l_rate=0.300, error=0.276
>epoch=10, l_rate=0.300, error=0.248
>epoch=11, l_rate=0.300, error=0.224
>epoch=12, l_rate=0.300, error=0.205
>epoch=13, l_rate=0.300, error=0.189
>epoch=14, l_rate=0.300, error=0.174
>epoch=15, l_rate=0.300, error=0.162
>epoch=16, l_rate=0.300, error=0.151
>epoch=17, l_rate=0.300, error=0.142
>epoch=18, l_rate=0.300, error=0.134
>epoch=19, l_rate=0.300, error=0.126
>epoch=20, l_rate=0.300, error=0.119
>epoch=21, l_rate=0.300, error=0.113
>epoch=22, l_rate=0.300, error=0.108
>epoch=23, l_rate=0.300, error=0.103
>epoch=24, l_rate=0.300, error=0.098
>epoch=25, l_rate=0.300, error=0.094
>epoch=26, l_rate=0.300, error=0.090
>epoch=27, 

# 3. Diabetes Prediction

In [6]:
# Logistic Regression on Diabetes Dataset
from random import seed
from random import randrange
from csv import reader
from math import exp

In [7]:
# Loading a csv file
def load_csv(filename):
    dataset = list()
    with open(filename,'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [8]:
# Converting string column to float
def str_column_to_float(dataset,column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [9]:
# Finding the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min,value_max])
    return minmax

In [10]:
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset,minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0])/(minmax[i][1] - minmax[i][0])

In [11]:
# split a dataset into k folds
def cross_validation_split(dataset,n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset)/n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [12]:
# calculating accuracy percentage
def accuracy_metric(actual,predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct+=1
    return correct/float(len(actual)) * 100.0

In [13]:
# Evaluating an algorithm using a cross validation split
def evaluate_algorithm(dataset,algorithm,n_folds,*args):
    folds = cross_validation_split(dataset,n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set,[])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set,test_set,*args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual,predicted)
        scores.append(accuracy)
    return scores

In [14]:
# Making a prediction with coefficients
def predict(row,coefficients):
    yhat = coefficients[0]
    for i in range(len(row)-1):
        yhat+= coefficients[i+1] * row[i]
    return 1.0/(1.0 + exp(-yhat))

In [15]:
# Estimating logistic regression coefficients using stochastic gradient descent
def coeffiients_sgd(train,l_rate,n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        for row in train:
            yhat = predict(row,coef)
            error = row[-1] - yhat
            coef[0] = coef[0] + l_rate * error * yhat * (1.0 - yhat)
            for i in  range(len(row)-1):
                coef[i+1] = coef[i+1] + l_rate * error * yhat * (1.0 - yhat) * row[i]
    return coef

In [16]:
# Logistic regression algorithm with stochastic gradient descent
def logistic_regression(train,test,l_rate,n_epoch):
    predictions = list()
    coef = coefficients_sgd(train,l_rate,n_epoch)
    for row in test:
        yhat = predict(row,coef)
        yhat = round(yhat)
        predictions.append(yhat)
    return predictions

In [17]:
# Testing the logistic regression on diabetes dataset
seed(1)

# loading and preparing data
filename = 'pima-indians-diabetes.data.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])):
    str_column_to_float(dataset,i)
    
# normalize
minmax = dataset_minmax(dataset)
normalize_dataset(dataset,minmax)

# evaluating algorithm
n_folds = 5
l_rate = 0.1
n_epochs = 100
scores = evaluate_algorithm(dataset,logistic_regression,n_folds,l_rate,n_epoch)
print('Scores : %s' %scores)
print('Mean Accuracy: %.3f'%(sum(scores)/float(len(scores))))

>epoch=0, l_rate=0.100, error=139.813
>epoch=1, l_rate=0.100, error=133.309
>epoch=2, l_rate=0.100, error=128.382
>epoch=3, l_rate=0.100, error=124.424
>epoch=4, l_rate=0.100, error=121.213
>epoch=5, l_rate=0.100, error=118.571
>epoch=6, l_rate=0.100, error=116.362
>epoch=7, l_rate=0.100, error=114.487
>epoch=8, l_rate=0.100, error=112.875
>epoch=9, l_rate=0.100, error=111.473
>epoch=10, l_rate=0.100, error=110.240
>epoch=11, l_rate=0.100, error=109.148
>epoch=12, l_rate=0.100, error=108.172
>epoch=13, l_rate=0.100, error=107.294
>epoch=14, l_rate=0.100, error=106.501
>epoch=15, l_rate=0.100, error=105.780
>epoch=16, l_rate=0.100, error=105.122
>epoch=17, l_rate=0.100, error=104.518
>epoch=18, l_rate=0.100, error=103.963
>epoch=19, l_rate=0.100, error=103.450
>epoch=20, l_rate=0.100, error=102.975
>epoch=21, l_rate=0.100, error=102.533
>epoch=22, l_rate=0.100, error=102.122
>epoch=23, l_rate=0.100, error=101.739
>epoch=24, l_rate=0.100, error=101.380
>epoch=25, l_rate=0.100, error=101.

>epoch=82, l_rate=0.100, error=94.925
>epoch=83, l_rate=0.100, error=94.894
>epoch=84, l_rate=0.100, error=94.863
>epoch=85, l_rate=0.100, error=94.832
>epoch=86, l_rate=0.100, error=94.803
>epoch=87, l_rate=0.100, error=94.775
>epoch=88, l_rate=0.100, error=94.747
>epoch=89, l_rate=0.100, error=94.720
>epoch=90, l_rate=0.100, error=94.694
>epoch=91, l_rate=0.100, error=94.668
>epoch=92, l_rate=0.100, error=94.643
>epoch=93, l_rate=0.100, error=94.619
>epoch=94, l_rate=0.100, error=94.595
>epoch=95, l_rate=0.100, error=94.572
>epoch=96, l_rate=0.100, error=94.550
>epoch=97, l_rate=0.100, error=94.528
>epoch=98, l_rate=0.100, error=94.507
>epoch=99, l_rate=0.100, error=94.486
>epoch=0, l_rate=0.100, error=142.570
>epoch=1, l_rate=0.100, error=134.535
>epoch=2, l_rate=0.100, error=128.794
>epoch=3, l_rate=0.100, error=124.254
>epoch=4, l_rate=0.100, error=120.620
>epoch=5, l_rate=0.100, error=117.662
>epoch=6, l_rate=0.100, error=115.213
>epoch=7, l_rate=0.100, error=113.154
>epoch=8, l_